<a href="https://colab.research.google.com/github/Long3309/CS221-Project/blob/main/Project_CS221.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Tải thư viện


In [1]:
from collections import defaultdict
import unicodedata as ud
import pandas as pd
import numpy as np
import math
import ast
import re
import nltk


In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)

# 2. Load dữ liệu

In [3]:
url = "https://raw.githubusercontent.com/Long3309/CS221-Project/main/dataset/sentences.txt"

In [16]:
!git clone https://github.com/Long3309/CS221-Project.git

Cloning into 'CS221-Project'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 20 (delta 5), reused 17 (delta 5), pack-reused 0
Unpacking objects: 100% (20/20), done.


In [27]:
!ls CS221-Project/dataset

gold.txt       test_gold.txt   train_gold.txt
sentences.txt  test_words.txt  train_words.txt


In [28]:
sentences = open('CS221-Project/dataset/sentences.txt', encoding='utf-8').readlines()
tokenize_sentences = [sentence.split(' ') for sentence in sentences]

In [21]:
print('Số lượng câu đã thu thập:', len(sentences))
sentences[0:2]

Số lượng câu đã thu thập: 60


['Pha lập công trên đã giúp Rashford giải hạn bàn thắng tại sân Old Trafford kéo dài 845 phút .\n',
 'Với 3 điểm có được trong trận đấu này , Quỷ đỏ đã leo lên vị trí thứ 2 trên bảng xếp hạng Premier League với 30 điểm , chỉ kém đội đầu bảng Liverpool 2 điểm .\n']

In [29]:
max_str = max(tokenize_sentences, key=len)
print('Câu có số từ nhiều nhât:', len(max_str))
' '.join(max_str)

Câu có số từ nhiều nhât: 46


'Những thực phẩm được chế biến như hun khói , thức ăn ngâm tẩm , muối , món ăn chứa lượng muối cao thường có tỷ lệ mắc ung thư dạ dày cao hơn những người có thói quen ăn uống nhạt và thanh đạm .\n'

In [30]:
min_str = min(tokenize_sentences, key=len)
print('Câu có số từ ít nhât:', len(min_str))
' '.join(min_str)

Câu có số từ ít nhât: 8


'Nhiều người có hoàn cảnh giống ông .\n'

# 3. Tách từ


## 3.1 Sử dụng thuật toán Longest-Matching

In [31]:
def syllablize(sentence):
    word = '\w+'
    non_word = '[^\w\s]'
    digits = '\d+([\.,_]\d+)+'
    
    patterns = []
    patterns.extend([word, non_word, digits])
    patterns = f"({'|'.join(patterns)})"
    
    sentence = ud.normalize('NFC', sentence)
    tokens = re.findall(patterns, sentence, re.UNICODE)
    return [token[0] for token in tokens]

In [32]:
def load_n_grams(path):
    with open(path, encoding='utf8') as f:
        words = f.read()
        words = ast.literal_eval(words)
    return words

In [34]:
def longest_matching(sentence, bi_grams, tri_grams):
    syllables = syllablize(sentence)
    syl_len = len(syllables)
    
    curr_id = 0
    word_list = []
    done = False
    
    while (curr_id < syl_len) and (not done):
        curr_word = syllables[curr_id]
        if curr_id >= syl_len - 1:
            word_list.append(curr_word)
            done = True
        else:
            next_word = syllables[curr_id + 1]
            pair_word = ' '.join([curr_word.lower(), next_word.lower()])
            if curr_id >= (syl_len - 2):
                if pair_word in bi_grams:
                    word_list.append('_'.join([curr_word, next_word]))
                    curr_id += 2
                else:
                    word_list.append(curr_word)
                    curr_id += 1
            else:
                next_next_word = syllables[curr_id + 2]
                triple_word = ' '.join([pair_word, next_next_word.lower()])
                if triple_word in tri_grams:
                    word_list.append('_'.join([curr_word, next_word, next_next_word]))
                    curr_id += 3
                elif pair_word in bi_grams:
                    word_list.append('_'.join([curr_word, next_word]))
                    curr_id += 2
                else:
                    word_list.append(curr_word)
                    curr_id += 1
    return word_list

In [36]:
bi_grams = load_n_grams('CS221-Project/resources/bi_grams.txt')
tri_grams = load_n_grams('CS221-Project/resources/tri_grams.txt')
longest_matching('nhưng sự thực hiện vẫn còn chưa phù hợp', bi_grams, tri_grams)

['nhưng', 'sự_thực', 'hiện', 'vẫn', 'còn', 'chưa', 'phù_hợp']

In [38]:
with open('CS221-Project/tokenize/longest_matching_tokens.txt', 'w', encoding='utf-8') as f:
    longest_matching_sentences = []
    for sentence in sentences:
        word_list = longest_matching(sentence, bi_grams, tri_grams)
        longest_matching_sentences.append(' '.join(word_list))
        for word in word_list: f.write(word + '\n')
        if sentence != sentences[-1]: f.write('\n')
    f.write('\n')
longest_matching_sentences[0:3]

['Pha lập_công trên đã giúp Rashford giải hạn bàn thắng tại sân Old Trafford kéo dài 845 phút .',
 'Với 3 điểm có được trong trận đấu này , Quỷ đỏ đã leo lên vị_trí thứ 2 trên bảng xếp_hạng Premier League với 30 điểm , chỉ kém đội đầu_bảng Liverpool 2 điểm .',
 'Tổng_thống đắc_cử Joe Biden được cho đang cân_nhắc việc cắt bỏ chương_trình hiện_đại hóa hạt_nhân trị_giá 1 . 000 tỷ USD do chính_quyền Tổng_thống đương_nhiệm Donald Trump đề_xuất .']

In [39]:
count_longest_matching_compounds = 0
for sentence in longest_matching_sentences:
    for word in sentence.split():
        if '_' in word: count_longest_matching_compounds += 1
print('Số lượng từ ghép khi tách từ bằng thuật toán Longest Matching:', count_longest_matching_compounds)

Số lượng từ ghép khi tách từ bằng thuật toán Longest Matching: 257


## 3.2 Sử dụng thư viện VNCoreNLP


1.   Cài đặt py-vncorenlp
2.   Khởi tạo đối tượng VNCoreNLP
3.   Xử lý đối tượng VNCoreNLP








In [40]:
!pip install py-vncorenlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.3 MB/s 
  Created wheel for py-vncorenlp: filename=py_vncorenlp-0.1.3-py3-none-any.whl size=4307 sha256=45d5733f0d7df96eb0576a2f7a019b3e51c28b1797b251715b9184873c347e6a
  Stored in directory: /root/.cache/pip/wheels/87/be/55/d5930c1d90a09832e9afd57a9e13801e3d16c88f5a19f777ae
Successfully built py-vncorenlp


In [41]:
import py_vncorenlp
py_vncorenlp.download_model(save_dir='./')
model = py_vncorenlp.VnCoreNLP(annotators=["wseg", "pos", "ner", "parse"], save_dir='./')

In [42]:
model.annotate_text(sentences[0])

{0: [{'index': 1,
   'wordForm': 'Pha',
   'posTag': 'N',
   'nerLabel': 'O',
   'head': 5,
   'depLabel': 'sub'},
  {'index': 2,
   'wordForm': 'lập_công',
   'posTag': 'V',
   'nerLabel': 'O',
   'head': 1,
   'depLabel': 'nmod'},
  {'index': 3,
   'wordForm': 'trên',
   'posTag': 'N',
   'nerLabel': 'O',
   'head': 1,
   'depLabel': 'nmod'},
  {'index': 4,
   'wordForm': 'đã',
   'posTag': 'R',
   'nerLabel': 'O',
   'head': 5,
   'depLabel': 'adv'},
  {'index': 5,
   'wordForm': 'giúp',
   'posTag': 'V',
   'nerLabel': 'O',
   'head': 0,
   'depLabel': 'root'},
  {'index': 6,
   'wordForm': 'Rashford',
   'posTag': 'Np',
   'nerLabel': 'B-PER',
   'head': 5,
   'depLabel': 'dob'},
  {'index': 7,
   'wordForm': 'giải',
   'posTag': 'V',
   'nerLabel': 'O',
   'head': 5,
   'depLabel': 'vmod'},
  {'index': 8,
   'wordForm': 'hạn',
   'posTag': 'N',
   'nerLabel': 'O',
   'head': 7,
   'depLabel': 'dob'},
  {'index': 9,
   'wordForm': 'bàn',
   'posTag': 'N',
   'nerLabel': 'O',
   'h